<a href="https://colab.research.google.com/github/LRLeite/Data-Analytics/blob/main/Explorando%20Dados%20com%20SQL/dvd_rental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 📀 **Resumo do Dataset DVD Rental (PostgreSQL)**

Este notebook Python explora o **dataset DVD Rental**, uma base de dados robusta projetada para simular as operações de uma locadora de DVDs. O dataset é estruturado em 15 tabelas interrelacionadas, incluindo exemplos como `film`, `actor`, `customer` e `rental`, que permitem uma análise abrangente do negócio.

Os dados estão armazenados no **Supabase**, uma plataforma de banco de dados com estrutura SQL baseada em PostgreSQL. As consultas são realizadas diretamente via Python, utilizando o **SQLAlchemy**, um driver/ORM compatível com PostgreSQL que permite a execução de comandos SQL.

Para garantir a segurança e a integridade das credenciais de acesso, este notebook integra duas ferramentas do ecossistema Google Cloud:

* **Autenticação Google Colab (`google.colab.auth`):** Utilizada para autenticar-se de forma segura no ambiente Google Colab, permitindo o acesso ao Secret Manager.

* **Google Cloud Secret Manager (`google.cloud.secretmanager`):** Garante o gerenciamento seguro das credenciais do banco de dados, protegendo informações sensíveis e evitando que sejam expostas diretamente no código-fonte do notebook.
<br>
---
**Fonte dos dados:** [dvd-rental](https://www.postgresqltutorial.com/postgresql-getting-started/postgresql-sample-database/)

**Documentação PostreSQL:** [Current Version (17)](https://www.postgresql.org/docs/current/)



# **Bibliotecas**

In [1]:
#Instalação de bibliotecas necessárias
#!pip install google-cloud-secret-manager google-auth
#!pip install sqlalchemy

In [2]:
#Importação de módulos
from google.colab import auth
from google.cloud import secretmanager
import pandas as pd
from sqlalchemy import create_engine

# **Conexão com o banco de dados SQL**

In [3]:
#Autenticação Google Cloud para acessar chaves do Secret Manager
auth.authenticate_user()
print("Autenticação Google Cloud concluída!")

Autenticação Google Cloud concluída!


In [4]:
#Carregamento de credenciais do Secret Manager
PROJECT_ID = "inner-topic-464213-p6"
client = secretmanager.SecretManagerServiceClient()

try:
    name_host = f"projects/{PROJECT_ID}/secrets/supabase-db-host/versions/latest"
    response_host = client.access_secret_version(name = name_host)
    DB_HOST = response_host.payload.data.decode("UTF-8")

    name_name = f"projects/{PROJECT_ID}/secrets/supabase-db-name/versions/latest"
    response_name = client.access_secret_version(name = name_name)
    DB_NAME = response_name.payload.data.decode("UTF-8")

    name_user = f"projects/{PROJECT_ID}/secrets/supabase-db-user/versions/latest"
    response_user = client.access_secret_version(name = name_user)
    DB_USER = response_user.payload.data.decode("UTF-8")

    name_password = f"projects/{PROJECT_ID}/secrets/supabase-db-password/versions/latest"
    response_password = client.access_secret_version(name = name_password)
    DB_PASSWORD = response_password.payload.data.decode("UTF-8")

    name_port = f"projects/{PROJECT_ID}/secrets/supabase-db-port/versions/latest"
    response_port = client.access_secret_version(name = name_port)
    DB_PORT = response_port.payload.data.decode("UTF-8")

    print("Credenciais do Secret Manager carregadas com sucesso!")

except Exception as e:
    print(f"Erro ao acessar segredos do Secret Manager: {e}")
    print("Verifique se o PROJECT_ID, nomes dos secrets e permissões IAM estão corretos.")

Credenciais do Secret Manager carregadas com sucesso!


In [5]:
#Criação da engine SQLAlchemy
engine = None

try:
    db_uri = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(db_uri)
    print("\nEngine do SQLAlchemy criada com sucesso usando credenciais do Secret Manager!")

except Exception as e:
    print(f"Erro ao criar a engine do SQLAlchemy: {e}")


Engine do SQLAlchemy criada com sucesso usando credenciais do Secret Manager!


# **Operação básicas**

In [6]:
#Selecionar o nome, sobremnome e e-mail dos clientes
query_1 = """
SELECT first_name, last_name, email FROM customer;
"""

df_1 = pd.read_sql_query(query_1, engine)
display(df_1)

,first_name,last_name,email
0,Jared,Ely,jared.ely@sakilacustomer.org
1,Mary,Smith,mary.smith@sakilacustomer.org
2,Patricia,Johnson,patricia.johnson@sakilacustomer.org
3,Linda,Williams,linda.williams@sakilacustomer.org
4,Barbara,Jones,barbara.jones@sakilacustomer.org
...,...,...,...
594,Terrence,Gunderson,terrence.gunderson@sakilacustomer.org
595,Enrique,Forsythe,enrique.forsythe@sakilacustomer.org
596,Freddie,Duggan,freddie.duggan@sakilacustomer.org
597,Wade,Delvalle,wade.delvalle@sakilacustomer.org


In [7]:
#Selecionar os diferentes tipos de avaliações que os filmes podem apresentar
query_2 = """
SELECT DISTINCT rating FROM film;
"""

df_2 = pd.read_sql_query(query_2, engine)
print(df_2)

  rating
0     PG
1  PG-13
2  NC-17
3      R
4      G


In [8]:
#Selecionar o nome, sobrenome e e-mail de Lois Butler
query_3 = """
SELECT first_name, last_name, email
FROM customer
WHERE first_name = 'Lois' AND last_name = 'Butler';
"""

df_3 = pd.read_sql_query(query_3, engine)
display(df_3)

,first_name,last_name,email
0,Lois,Butler,lois.butler@sakilacustomer.org


In [9]:
#Selecionar a descrição do filme "Outlaw Hanky"
query_4 = """
SELECT description
FROM film
WHERE title = 'Outlaw Hanky';
"""

df_4 = pd.read_sql_query(query_4, engine)
pd.set_option('display.max_colwidth', None)
display(df_4)

,description
0,A Thoughtful Story of a Astronaut And a Composer who must Conquer a Dog in The Sahara Desert


# **Manipulação de datas**

In [10]:
#Verificar os dias em que houveram pagamento no 1° trimestre
query_5 = """
SELECT
  DISTINCT(CAST(EXTRACT(month FROM payment_date) AS INTEGER)) AS payment_month,
  TO_CHAR(payment_date, 'MONTH') AS payment_monthName,
  payment_date::date AS payment_date,
  TO_CHAR(payment_date, 'DD-mon-YYYY') AS formatted_date
FROM payment
WHERE EXTRACT(QUARTER FROM payment_date) = 1
ORDER BY payment_month
"""

df_5 = pd.read_sql_query(query_5, engine)
print(df_5)

    payment_month payment_monthname payment_date formatted_date
0               2         FEBRUARY    2007-02-14    14-feb-2007
1               2         FEBRUARY    2007-02-15    15-feb-2007
2               2         FEBRUARY    2007-02-16    16-feb-2007
3               2         FEBRUARY    2007-02-17    17-feb-2007
4               2         FEBRUARY    2007-02-18    18-feb-2007
5               2         FEBRUARY    2007-02-19    19-feb-2007
6               2         FEBRUARY    2007-02-20    20-feb-2007
7               2         FEBRUARY    2007-02-21    21-feb-2007
8               3         MARCH       2007-03-01    01-mar-2007
9               3         MARCH       2007-03-02    02-mar-2007
10              3         MARCH       2007-03-16    16-mar-2007
11              3         MARCH       2007-03-17    17-mar-2007
12              3         MARCH       2007-03-18    18-mar-2007
13              3         MARCH       2007-03-19    19-mar-2007
14              3         MARCH       20

# **Funções de agregação**

In [11]:
#Calcular valores agregados da tabela payment
query_6 = """
SELECT
    SUM(amount) AS total_amount,
    AVG(amount) AS average_amount,
    MIN(amount) AS min_amount,
    MAX(amount) AS max_amount
FROM payment;
"""

df_6 = pd.read_sql_query(query_6, engine)
print(df_6)

   total_amount  average_amount  min_amount  max_amount
0      61312.04        4.200606         0.0       11.99


In [12]:
#Calcular custo percentual dos filmes
query_7 = """
SELECT
    title,
    ROUND(rental_rate / replacement_cost, 2)*100 AS custo_percent
FROM film;
"""

df_7 = pd.read_sql_query(query_7, engine)
print(df_7)

                 title  custo_percent
0      Chamber Italian           33.0
1     Grosse Wonderful           25.0
2      Airport Pollock           31.0
3    Bright Encounters           38.0
4     Academy Dinosaur            5.0
..                 ...            ...
995     Young Language           10.0
996         Youth Kick            7.0
997       Zhivago Core            9.0
998  Zoolander Fiction           10.0
999          Zorro Ark           26.0

[1000 rows x 2 columns]


# **Manipulação de strings**

In [13]:
#Gerar nome completo e e-mail fictício
query_8 = """
SELECT
    customer_id,
    first_name || ' ' || last_name as full_name,
    email,
    LOWER(LEFT(first_name, 1)) || '.' || LOWER(last_name) || '@example.com' AS new_email
FROM customer;
"""

df_8 = pd.read_sql_query(query_8, engine)
display(df_8)

,customer_id,full_name,email,new_email
0,524,Jared Ely,jared.ely@sakilacustomer.org,j.ely@example.com
1,1,Mary Smith,mary.smith@sakilacustomer.org,m.smith@example.com
2,2,Patricia Johnson,patricia.johnson@sakilacustomer.org,p.johnson@example.com
3,3,Linda Williams,linda.williams@sakilacustomer.org,l.williams@example.com
4,4,Barbara Jones,barbara.jones@sakilacustomer.org,b.jones@example.com
...,...,...,...,...
594,595,Terrence Gunderson,terrence.gunderson@sakilacustomer.org,t.gunderson@example.com
595,596,Enrique Forsythe,enrique.forsythe@sakilacustomer.org,e.forsythe@example.com
596,597,Freddie Duggan,freddie.duggan@sakilacustomer.org,f.duggan@example.com
597,598,Wade Delvalle,wade.delvalle@sakilacustomer.org,w.delvalle@example.com


# **GROUP BY, HAVING, ORDER BY e LIMIT**




In [14]:
#Selecionar os 10 clientes com maior gasto total
query_9 = """
SELECT customer_id, SUM(amount) AS total_spent
FROM payment
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 10;
"""

df_9 = pd.read_sql_query(query_9, engine)
print(df_9)

   customer_id  total_spent
0          148       211.55
1          526       208.58
2          178       194.61
3          137       191.62
4          144       189.60
5          459       183.63
6          181       167.67
7          410       167.62
8          236       166.61
9          403       162.67


In [15]:
#Verificar qual funcionário lida com a maioria dos pagamentos
query_10 = """
SELECT staff_id, COUNT(*) AS payment_count
FROM payment
GROUP BY staff_id
ORDER BY payment_count DESC
LIMIT 1;
"""

df_10 = pd.read_sql_query(query_10, engine)
print(df_10)

   staff_id  payment_count
0         2           7304


In [16]:
#Verificar a média do custo de reposição para os filmes com avaliação 'G', 'PG' e 'R'
query_11 = """
SELECT rating, ROUND(AVG(replacement_cost), 2) AS avg_replacement_cost
FROM film
WHERE rating IN ('G', 'PG', 'R')
GROUP BY rating;
"""

df_11 = pd.read_sql_query(query_11, engine)
print(df_11)

  rating  avg_replacement_cost
0     PG                 18.96
1      R                 20.23
2      G                 20.12


In [17]:
#Verificar os clientes que tiveram mais de 39 transações
query_12 = """
SELECT customer_id, COUNT(*) AS transaction_count
FROM payment
GROUP BY customer_id
HAVING COUNT(*) > 39
ORDER BY COUNT(*) DESC;
"""

df_12 = pd.read_sql_query(query_12, engine)
print(df_12)

   customer_id  transaction_count
0          148                 45
1          526                 42
2          144                 40


In [18]:
#Selecionar os clientes que gastaram mais de $100 com o funcionário id 2
query_13 = """
SELECT customer_id, SUM(amount) AS total_spent
FROM payment
WHERE staff_id = 2
GROUP BY customer_id
HAVING SUM(amount) > 100
ORDER BY SUM(amount) DESC;
"""

df_13 = pd.read_sql_query(query_13, engine)
display(df_13)

,customer_id,total_spent
0,187,110.81
1,148,110.78
2,211,108.77
3,522,102.80
4,526,101.78


# **JOINs**

## **LEFT JOIN**

In [19]:
#Selecionar os filmes que nunca foram alugados
query_14 = """
SELECT distinct(f.title) AS film_title
FROM film AS f
LEFT JOIN inventory AS i ON f.film_id = i.film_id
LEFT JOIN rental AS r ON i.inventory_id = r.inventory_id
WHERE r.rental_date IS NULL
ORDER BY film_title;
"""

df_14 = pd.read_sql_query(query_14, engine)
film_titles = df_14['film_title'].tolist()

for i in range(0, len(film_titles), 5):
    print(film_titles[i:i+5])

['Academy Dinosaur', 'Alice Fantasia', 'Apollo Teen', 'Argonauts Town', 'Ark Ridgemont']
['Arsenic Independence', 'Boondock Ballroom', 'Butch Panther', 'Catch Amistad', 'Chinatown Gladiator']
['Chocolate Duck', 'Commandments Express', 'Crossing Divorce', 'Crowds Telemark', 'Crystal Breaking']
['Dazed Punk', 'Deliverance Mulholland', 'Firehouse Vietnam', 'Floats Garden', 'Frankenstein Stranger']
['Gladiator Westward', 'Gump Date', 'Hate Handicap', 'Hocus Frida', 'Kentuckian Giant']
['Kill Brotherhood', 'Muppet Mile', 'Order Betrayed', 'Pearl Destiny', 'Perdition Fargo']
['Psycho Shrunk', 'Raiders Antitrust', 'Rainbow Shock', 'Roof Champion', 'Sister Freddy']
['Sky Miracle', 'Suicides Silence', 'Tadpole Park', 'Treasure Command', 'Villain Desperate']
['Volume House', 'Wake Jaws', 'Walls Artist']


## **RIGH JOIN**

In [20]:
#Selecionar as 10 categorias que possuem mais filmes
query_15 = """
SELECT c.name AS categoria, COUNT(fc.film_id) AS qtd_filmes
FROM category AS c
RIGHT JOIN film_category AS fc ON c.category_id = fc.category_id
GROUP BY categoria
ORDER BY qtd_filmes DESC
LIMIT 10;
"""

df_15 = pd.read_sql_query(query_15, engine)
display(df_15)

,categoria,qtd_filmes
0,Sports,74
1,Foreign,73
2,Family,69
3,Documentary,68
4,Animation,66
5,Action,64
6,New,63
7,Drama,62
8,Sci-Fi,61
9,Games,61


## **FULL OUTER JOIN**

In [21]:
#Identificar atores que não atuaram em nenhum filme OU os filmes que não têm nenhum ator associado
query_16 = """
SELECT
    a.first_name || ' ' || a.last_name AS actor_name,
    f.title AS film_title
FROM
    actor AS a
FULL JOIN
    film_actor AS fa ON a.actor_id = fa.actor_id
FULL JOIN
    film AS f ON fa.film_id = f.film_id
WHERE
    a.actor_id IS NULL OR f.film_id IS NULL;
"""

df_16 = pd.read_sql_query(query_16, engine)
display(df_16)

,actor_name,film_title
0,None,Slacker Liaisons
1,None,Flight Lies
2,None,Drumline Cyclone


## **INNER JOIN**

In [22]:
#Selecionar os e-mails dos clientes que moram na Califórnia
query_17 = """
SELECT district, email FROM customer
INNER JOIN address ON customer.address_id = address.address_id
WHERE district = 'California';
"""

df_17 = pd.read_sql_query(query_17, engine)
display(df_17)

,district,email
0,California,patricia.johnson@sakilacustomer.org
1,California,betty.white@sakilacustomer.org
2,California,alice.stewart@sakilacustomer.org
3,California,rosa.reynolds@sakilacustomer.org
4,California,renee.lane@sakilacustomer.org
5,California,kristin.johnston@sakilacustomer.org
6,California,cassandra.walters@sakilacustomer.org
7,California,jacob.lance@sakilacustomer.org
8,California,rene.mcalister@sakilacustomer.org


In [23]:
#Encontrar os filmes em que Lisa Monroe participou
query_18 = """
SELECT first_name || ' ' || last_name AS name, title FROM actor
INNER JOIN film_actor ON actor.actor_id = film_actor.actor_id
INNER JOIN film ON film_actor.film_id = film.film_id
WHERE(first_name || ' ' || last_name) = 'Lisa Monroe'
"""

df_18 = pd.read_sql_query(query_18, engine)
print(df_18)

           name               title
0   Lisa Monroe   Anything Savannah
1   Lisa Monroe        Arabia Dogma
2   Lisa Monroe  Butterfly Chocolat
3   Lisa Monroe         Chitty Lock
4   Lisa Monroe       Club Graffiti
5   Lisa Monroe       Coast Rainbow
6   Lisa Monroe         Crow Grease
7   Lisa Monroe       Crusade Honey
8   Lisa Monroe    Effect Gladiator
9   Lisa Monroe   Fiction Christmas
10  Lisa Monroe       Hanky October
11  Lisa Monroe       Jericho Mulan
12  Lisa Monroe    Lesson Cleopatra
13  Lisa Monroe        Lover Truman
14  Lisa Monroe       Mod Secretary
15  Lisa Monroe         Quills Bull
16  Lisa Monroe        River Outlaw
17  Lisa Monroe      Roots Remember
18  Lisa Monroe        Sassy Packer
19  Lisa Monroe   Vacation Boondock
20  Lisa Monroe         Wild Apollo
21  Lisa Monroe           Won Dares
22  Lisa Monroe           Zorro Ark


In [24]:
#Selecionar os consumidores que alugaram o mesmo filme mais de uma vez
  #tabela customer: Contém dados do cliente (PK: customer_id). customer_id é FK em rental.
  #tabela rental: Registra aluguéis (PK: rental_id). Contém FKs customer_id e inventory_id.
  #tabela inventory: Detalhes das cópias de filmes (PK: inventory_id). film_id é FK para a tabela film.
  #tabela film: Detalhes dos filmes (PK: film_id). Contém o título do filme.

query_19 = """
SELECT c.first_name, c.last_name, f.title, COUNT(r.rental_id) AS num_alugueis
FROM customer AS c
INNER JOIN rental AS r ON c.customer_id = r.customer_id
INNER JOIN inventory AS i ON r.inventory_id = i.inventory_id
INNER JOIN film AS f ON i.film_id = f.film_id
GROUP BY c.customer_id, f.film_id
HAVING COUNT(r.rental_id) > 1
ORDER BY num_alugueis DESC;
"""

df_19 = pd.read_sql_query(query_19, engine)
display(df_19)

,first_name,last_name,title,num_alugueis
0,Thelma,Murray,Disciple Mother,3
1,Randy,Gaither,Detective Vision,3
2,George,Linton,Caddyshack Jedi,3
3,Edward,Baugh,Flatliners Killer,3
4,Wallace,Slone,Fiction Christmas,2
...,...,...,...,...
207,Duane,Tubbs,Strangers Graffiti,2
208,Johnnie,Chisholm,California Birds,2
209,Tracey,Barrett,Trip Newton,2
210,Courtney,Day,Driver Annie,2


## **Self-Join**

In [25]:
#Listar todos os pares de atores que atuaram juntos em pelo menos dois filmes diferentes
query_20 = """
SELECT
    a1.first_name || ' ' || a1.last_name AS actor1_full_name,
    a2.first_name || ' ' || a2.last_name AS actor2_full_name,
    COUNT(DISTINCT fa1.film_id) AS filmes_em_comum
FROM
    film_actor AS fa1
INNER JOIN
    film_actor AS fa2 ON fa1.film_id = fa2.film_id -- Atores no MESMO filme
INNER JOIN
    actor AS a1 ON fa1.actor_id = a1.actor_id -- Junta com a tabela actor para o nome do Ator 1
INNER JOIN
    actor AS a2 ON fa2.actor_id = a2.actor_id -- Junta com a tabela actor para o nome do Ator 2
WHERE
    fa1.actor_id < fa2.actor_id -- Garante que sejam atores DIFERENTES e evita duplicatas (A,B e B,A)
GROUP BY
    a1.actor_id, a2.actor_id, a1.first_name, a1.last_name, a2.first_name, a2.last_name
HAVING
    COUNT(DISTINCT fa1.film_id) > 1; -- Atuaram juntos em MAIS de um filme
"""

df_20 = pd.read_sql_query(query_20, engine)
display(df_20)

,actor1_full_name,actor2_full_name,filmes_em_comum
0,Penelope Guiness,Jennifer Davis,4
1,Penelope Guiness,Bette Nicholson,2
2,Penelope Guiness,Christian Gable,2
3,Penelope Guiness,Lucille Tracy,3
4,Penelope Guiness,Cameron Streep,2
...,...,...,...
3478,Burt Temple,Thora Temple,2
3479,Jayne Silverstone,Reese West,2
3480,Bela Walken,Mary Keitel,2
3481,Reese West,Mary Keitel,2


# **Condicionais**

## **CASE**

In [26]:
#Categorização de Duração de Filmes
query_21 = """
SELECT title, length,
CASE
        WHEN length <= 60 THEN 'curta'
ELSE 'longa'
END AS categoria
FROM film
"""

df_21 = pd.read_sql_query(query_21, engine)
display(df_21)

,title,length,categoria
0,Chamber Italian,117,longa
1,Grosse Wonderful,49,curta
2,Airport Pollock,54,curta
3,Bright Encounters,73,longa
4,Academy Dinosaur,86,longa
...,...,...,...
995,Young Language,183,longa
996,Youth Kick,179,longa
997,Zhivago Core,105,longa
998,Zoolander Fiction,101,longa


In [27]:
#Adicionar bônus por Classificação de Filme
query_22 = """
SELECT title, rating, rental_rate,
CASE
    WHEN rating IN ('G', 'PG') THEN ROUND(rental_rate * 0.10, 2)
    WHEN rating = 'PG-13' THEN ROUND(rental_rate * 0.15, 2)
    WHEN rating IN ('R', 'NC-17') THEN ROUND(rental_rate * 0.20, 2)
ELSE  0.00
END AS bonus
FROM film
"""

df_22 = pd.read_sql_query(query_22, engine)
display(df_22)

,title,rating,rental_rate,bonus
0,Chamber Italian,NC-17,4.99,1.00
1,Grosse Wonderful,R,4.99,1.00
2,Airport Pollock,R,4.99,1.00
3,Bright Encounters,PG-13,4.99,0.75
4,Academy Dinosaur,PG,0.99,0.10
...,...,...,...,...
995,Young Language,G,0.99,0.10
996,Youth Kick,NC-17,0.99,0.20
997,Zhivago Core,NC-17,0.99,0.20
998,Zoolander Fiction,R,2.99,0.60


## **COALESCE**

In [28]:
#Selecionar consumidores e seus respectivos e-mails e substituir os e-mails nulls por 'indisponivel'
query_23 = """
SELECT
    first_name || ' ' || last_name AS full_name,
    COALESCE(email, 'indisponivel') AS email_contato
FROM
    customer;
"""

df_23 = pd.read_sql_query(query_23, engine)
display(df_23)

,full_name,email_contato
0,Jared Ely,jared.ely@sakilacustomer.org
1,Mary Smith,mary.smith@sakilacustomer.org
2,Patricia Johnson,patricia.johnson@sakilacustomer.org
3,Linda Williams,linda.williams@sakilacustomer.org
4,Barbara Jones,barbara.jones@sakilacustomer.org
...,...,...
594,Terrence Gunderson,terrence.gunderson@sakilacustomer.org
595,Enrique Forsythe,enrique.forsythe@sakilacustomer.org
596,Freddie Duggan,freddie.duggan@sakilacustomer.org
597,Wade Delvalle,wade.delvalle@sakilacustomer.org


## **NULLIF**

In [29]:
#Substituir observações que possuem como valor espaço(s) em branco por null
query_24 = """
SELECT
    first_name || ' ' || last_name AS full_name,
    NULLIF(TRIM(email), '') AS email_limpo
FROM
    customer;
"""

df_24 = pd.read_sql_query(query_24, engine)
display(df_24)

,full_name,email_limpo
0,Jared Ely,jared.ely@sakilacustomer.org
1,Mary Smith,mary.smith@sakilacustomer.org
2,Patricia Johnson,patricia.johnson@sakilacustomer.org
3,Linda Williams,linda.williams@sakilacustomer.org
4,Barbara Jones,barbara.jones@sakilacustomer.org
...,...,...
594,Terrence Gunderson,terrence.gunderson@sakilacustomer.org
595,Enrique Forsythe,enrique.forsythe@sakilacustomer.org
596,Freddie Duggan,freddie.duggan@sakilacustomer.org
597,Wade Delvalle,wade.delvalle@sakilacustomer.org


# **SubQuery**

In [30]:
#Selecionar s consumidores que gastaram mais de $100 na loja
#O valor gasto por consumidor está na tabela payment e as demais informações estão na tabela customer
query_25 = """
SELECT customer_id, first_name || ' ' || last_name AS full_name, email
FROM customer
WHERE customer_id IN (
    SELECT customer_id
    FROM payment
    GROUP BY customer_id
    HAVING SUM(amount) > 100
);
"""

df_25 = pd.read_sql_query(query_25, engine)
display(df_25)

,customer_id,full_name,email
0,1,Mary Smith,mary.smith@sakilacustomer.org
1,2,Patricia Johnson,patricia.johnson@sakilacustomer.org
2,3,Linda Williams,linda.williams@sakilacustomer.org
3,5,Elizabeth Brown,elizabeth.brown@sakilacustomer.org
4,7,Maria Miller,maria.miller@sakilacustomer.org
...,...,...,...
291,591,Kent Arsenault,kent.arsenault@sakilacustomer.org
292,592,Terrance Roush,terrance.roush@sakilacustomer.org
293,593,Rene Mcalister,rene.mcalister@sakilacustomer.org
294,594,Eduardo Hiatt,eduardo.hiatt@sakilacustomer.org


In [31]:
#Selecionar o nome completo e e-mail dos consumidores que gastaram acima da média da loja,
#utilizando tabela temporária para mostrar também o valor gasto por cada consumidor
#O valor gasto por consumidor está na tabela payment e as demais informações estão na tabela customer
query_26 = '''
WITH CustomerSpending AS (
        SELECT customer_id, SUM(amount) as total_spent
        FROM payment
        GROUP BY customer_id
        HAVING SUM(amount) > (SELECT AVG(amount) FROM payment) -- Filtra os consumidores que gastaram acima da média
)
SELECT first_name || ' ' || last_name AS full_name, email, total_spent
FROM customer
INNER JOIN CustomerSpending on customer.customer_id = CustomerSpending.customer_id
ORDER BY total_spent DESC
'''

df_26 = pd.read_sql_query(query_26, engine)
display(df_26)

,full_name,email,total_spent
0,Eleanor Hunt,eleanor.hunt@sakilacustomer.org,211.55
1,Karl Seal,karl.seal@sakilacustomer.org,208.58
2,Marion Snyder,marion.snyder@sakilacustomer.org,194.61
3,Rhonda Kennedy,rhonda.kennedy@sakilacustomer.org,191.62
4,Clara Shaw,clara.shaw@sakilacustomer.org,189.60
...,...,...,...
594,Tiffany Jordan,tiffany.jordan@sakilacustomer.org,49.88
595,Anthony Schwab,anthony.schwab@sakilacustomer.org,47.85
596,Caroline Bowman,caroline.bowman@sakilacustomer.org,37.87
597,Leona Obrien,leona.obrien@sakilacustomer.org,32.90


# **VIEW**

In [34]:
#View que foi criada no Supabase (banco de dados Postgre)
'''
CREATE OR REPLACE VIEW filmes_detalhes_view AS
SELECT
    f.film_id,
    f.title AS film_title,
    f.description AS film_description,
    f.release_year,
    l.name AS film_language,
    f.length AS film_length_minutes,
    f.replacement_cost AS film_replacement_cost_usd,
    f.rating AS film_rating_mpaa,
    ca.name AS film_category_name
FROM
    film AS f
INNER JOIN
    language AS l ON f.language_id = l.language_id
INNER JOIN
    film_category AS fc ON f.film_id = fc.film_id
INNER JOIN
    category AS ca ON fc.category_id = ca.category_id;
'''

#Consultar view
query_view = "SELECT * FROM filmes_detalhes_view LIMIT 5;"

df_query_view = pd.read_sql_query(query_view, engine)
display(df_query_view)

,film_id,film_title,film_description,release_year,film_language,film_length_minutes,film_replacement_cost_usd,film_rating_mpaa,film_category_name
0,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,English,86,20.99,PG,Documentary
1,2,Ace Goldfinger,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,English,48,12.99,G,Horror
2,3,Adaptation Holes,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,English,50,18.99,NC-17,Documentary
3,4,Affair Prejudice,A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank,2006,English,117,26.99,G,Horror
4,5,African Egg,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,2006,English,130,22.99,G,Family


**<h1>Encerrar conexão com o banco de dados</h1>**

In [33]:
#Disposição da engine SQLAlchemy ao final
print("\n--- Fechando Conexão ---")
if engine:
    engine.dispose()
    print("Engine SQLAlchemy disposed.")
print("Notebook finalizado.")


--- Fechando Conexão ---
Engine SQLAlchemy disposed.
Notebook finalizado.
